In [1]:
#############################################################################
# zlib License
#
# (C) 2023 Zach Flowers, Murtaza Safdari <musafdar@cern.ch>
#
# This software is provided 'as-is', without any express or implied
# warranty.  In no event will the authors be held liable for any damages
# arising from the use of this software.
#
# Permission is granted to anyone to use this software for any purpose,
# including commercial applications, and to alter it and redistribute it
# freely, subject to the following restrictions:
#
# 1. The origin of this software must not be misrepresented; you must not
#    claim that you wrote the original software. If you use this software
#    in a product, an acknowledgment in the product documentation would be
#    appreciated but is not required.
# 2. Altered source versions must be plainly marked as such, and must not be
#    misrepresented as being the original software.
# 3. This notice may not be removed or altered from any source distribution.
#############################################################################

# Imports

In [4]:
#%%
%matplotlib inline
import matplotlib.pyplot as plt
import logging
import i2c_gui
import i2c_gui.chips
from i2c_gui.usb_iss_helper import USB_ISS_Helper
from i2c_gui.fpga_eth_helper import FPGA_ETH_Helper
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable
import time
from tqdm import tqdm
# from i2c_gui.chips.etroc2_chip import register_decoding
import os, sys
import multiprocessing
os.chdir(f'/home/{os.getlogin()}/ETROC2/ETROC_DAQ')
import run_script
import parser_arguments
import importlib
importlib.reload(run_script)
import datetime
import pandas
from pathlib import Path
import subprocess
import sqlite3
from notebooks.notebook_helpers import *
from fnmatch import fnmatch
import scipy.stats as stats
from math import ceil

# Set defaults

In [5]:
hostname = "192.168.2.3"

polarity = "0x0027" # or "0x0022"
# polarity = "0x000f"

# Run One Time DAQ to Set FPGA Firmware

In [7]:
parser = parser_arguments.create_parser()
# (options, args) = parser.parse_args(args=f"-f --useIPC --hostname {hostname} -t 15 -o clear_fifo -v -w -s 0x0000 -p {polarity} -d 0xbb00 -a 0x000b --clear_fifo --clear_error".split())
(options, args) = parser.parse_args(args=f"-f --useIPC --hostname {hostname} -t 15 -o clear_fifo -v -w -s 0x0000 -p {polarity} -d 0x1800 -a 0x0001 --clear_fifo --check_valid_data_start --start_DAQ_pulse --stop_DAQ_pulse".split())
IPC_queue = multiprocessing.Queue()
process = multiprocessing.Process(target=run_script.main_process, args=(IPC_queue, options, None))
process.start()

IPC_queue.put('memoFC Start Triggerbit QInj L1A BCR')
while not IPC_queue.empty():
    pass
time.sleep(10)
IPC_queue.put('stop DAQ')
IPC_queue.put('memoFC Triggerbit')
while not IPC_queue.empty():
    pass
IPC_queue.put('allow threads to exit')
process.join()

del IPC_queue, process, parser

start main process
Setting firmware...
writing: 0b1 into register 15
Written into Reg 7:  0000000000000000
Time (s) for counting stats in FPGA:  000000 0
Enable Channel Trigger Bit 1 clock delay:  0000


Written into Reg 8:  0001100000000000
Enhance data LED (LED Page 011):  1
Enable L1A upon Rx trigger bit :  0
10 bit delay (trigger bit->L1A):  0000000000 0


Written into Reg 11:  0000000000000000
Written into Reg 12:  0000000000000000


Written into Reg 13:  0000000000000000
Data Rate              :  00
LED pages              :  000
Testmode               :  0
Timestamp (active low) :  0
DEV Status Page        :  00


Written into Reg 14:  0000000000100111
Add Ethernet Filler:  1
Enable Debug mode  :  0
Enable Dumping mode:  0
Polarity           :  1
Disable GTX        :  1
Enable AutoSync    :  1


Written into Reg 15  :  0000000000000001
Channel Enable       :  0001
WS Trig Stop Delay   :  000
Enable WS Trigger    :  0
Triple Trigger       :  0
Trigger OR Logic     :  0
Enable Glob

In [ ]:
#  --start_DAQ_pulse --stop_DAQ_pulse

#### check LEDs

In [ ]:
parser = parser_arguments.create_parser()
(options, args) = parser.parse_args(args=f"-f --useIPC --hostname {hostname} -t 15 -o LED_check -v -w -s 0x0000 -p {polarity} -d 0xbb00 -a 0x000b --nodaq".split())
IPC_queue = multiprocessing.Queue()
process = multiprocessing.Process(target=run_script.main_process, args=(IPC_queue, options, f'main_process_checkLEDs_0'))
process.start()
time.sleep(10)
process.join()

del IPC_queue, process, parser

parser = parser_arguments.create_parser()
(options, args) = parser.parse_args(args=f"-f --useIPC --hostname {hostname} -t 15 -o LED_check -v -w -s 0x0004 -p {polarity} -d 0xbb00 -a 0x000b --nodaq".split())
IPC_queue = multiprocessing.Queue()
process = multiprocessing.Process(target=run_script.main_process, args=(IPC_queue, options, f'main_process_checkLEDs_4'))
process.start()
time.sleep(10)
process.join()

del IPC_queue, process, parser

parser = parser_arguments.create_parser()
(options, args) = parser.parse_args(args=f"-f --useIPC --hostname {hostname} -t 15 -o LED_check -v -w -s 0x000C -p {polarity} -d 0xbb00 -a 0x000b --nodaq".split())
IPC_queue = multiprocessing.Queue()
process = multiprocessing.Process(target=run_script.main_process, args=(IPC_queue, options, f'main_process_checkLEDs_C'))
process.start()
time.sleep(10)
process.join()

del IPC_queue, process, parser

#### check LEDs (old firmware)

In [9]:
parser = parser_arguments.create_parser()
(options, args) = parser.parse_args(args=f"-f --useIPC --hostname {hostname} -t 15 -o LED_check -v -w -s 0x0000 -p {polarity} -d 0xb800 -a 0x00bb --nodaq".split())
IPC_queue = multiprocessing.Queue()
process = multiprocessing.Process(target=run_script.main_process, args=(IPC_queue, options, f'main_process_checkLEDs_0'))
process.start()
time.sleep(10)
process.join()

del IPC_queue, process, parser

In [10]:
parser = parser_arguments.create_parser()
(options, args) = parser.parse_args(args=f"-f --useIPC --hostname {hostname} -t 15 -o LED_check -v -w -s 0x0004 -p {polarity} -d 0xb800 -a 0x00bb --nodaq".split())
IPC_queue = multiprocessing.Queue()
process = multiprocessing.Process(target=run_script.main_process, args=(IPC_queue, options, f'main_process_checkLEDs_4'))
process.start()
time.sleep(10)
process.join()

del IPC_queue, process, parser

In [11]:
parser = parser_arguments.create_parser()
(options, args) = parser.parse_args(args=f"-f --useIPC --hostname {hostname} -t 15 -o LED_check -v -w -s 0x000C -p {polarity} -d 0xb800 -a 0x00bb --nodaq".split())
IPC_queue = multiprocessing.Queue()
process = multiprocessing.Process(target=run_script.main_process, args=(IPC_queue, options, f'main_process_checkLEDs_C'))
process.start()
time.sleep(10)
process.join()

del IPC_queue, process, parser

#### Without clear_fifo

In [ ]:
# os.system("rm -r ../ETROC-Data/2023-11-02_Array_Test_Results/CanBeRemoved_Board0_NoLinkCheck/")
parser = parser_arguments.create_parser()
(options, args) = parser.parse_args(args=f"-f --useIPC --hostname {hostname} -t 15 -o CanBeRemoved_Board013_NoLinkCheck -v -w -s 0x0000 -p {polarity} -d 0xb800 -a 0x00bb --check_valid_data_start --clear_error".split())
IPC_queue = multiprocessing.Queue()
# process = multiprocessing.Process(target=run_script.main_process, args=(IPC_queue, options, f'main_process_Start_LEDs_Board013_NoLinkCheck'))
process = multiprocessing.Process(target=run_script.main_process, args=(IPC_queue, options, None))
process.start()

IPC_queue.put('memoFC Start Triggerbit QInj L1A BCR')
while not IPC_queue.empty():
    pass
time.sleep(10)
IPC_queue.put('stop DAQ')
IPC_queue.put('memoFC Triggerbit')
while not IPC_queue.empty():
    pass
IPC_queue.put('allow threads to exit')
process.join()

del IPC_queue, process, parser

In [ ]:
parser = parser_arguments.create_parser()
(options, args) = parser.parse_args(args=f"-f --useIPC --hostname {hostname} -t 15 -o clear_fifo -v -w -s 0x0004 -p {polarity} -d 0x3800 -a 0x0033 --clear_fifo --clear_error".split())
IPC_queue = multiprocessing.Queue()
process = multiprocessing.Process(target=run_script.main_process, args=(IPC_queue, options, None))
process.start()

IPC_queue.put('memoFC Start Triggerbit QInj L1A BCR')
while not IPC_queue.empty():
    pass
time.sleep(10)
IPC_queue.put('stop DAQ')
IPC_queue.put('memoFC Triggerbit')
while not IPC_queue.empty():
    pass
IPC_queue.put('allow threads to exit')
process.join()

del IPC_queue, process, parser

# Run DAQ with Self-Trigger

## Find the right delay (use the middle of the three success modes, where both Qinj edges are caught)

### Board 0 Only

In [ ]:
for inum in range(475, 490, 1):
    trigger_bit_delay = inum
    print(inum)
    parser = parser_arguments.create_parser()
    (options, args) = parser.parse_args(args=f"-f --useIPC --hostname {hostname} -t 20 -o CanBeRemoved_Board0_LinkCheck_selfTrigger_delay{inum} -v -w -s 0x0000 -p {polarity} -d {int('000111'+format(inum, '010b'), base=2)} -a 0x0011".split())
    IPC_queue = multiprocessing.Queue()
    process = multiprocessing.Process(target=run_script.main_process, args=(IPC_queue, options, f'main_process_Start_LEDs_Board0_LinkCheck_selfTrigger_delay{inum}'))
    process.start()

    IPC_queue.put('memoFC Start Triggerbit QInj BCR')
    while not IPC_queue.empty():
        pass
    time.sleep(10)
    IPC_queue.put('stop DAQ')
    IPC_queue.put('memoFC Triggerbit')
    while not IPC_queue.empty():
        pass
    IPC_queue.put('allow threads to exit')
    process.join()

    del IPC_queue, process, parser

#### test delay from previous cell

In [9]:
trigger_bit_delay = int('000111'+format(485, '010b'), base=2)
parser = parser_arguments.create_parser()
(options, args) = parser.parse_args(args=f"-f --useIPC --hostname {hostname} -t 15 -o CanBeRemoved_Board0_selfTrigger -v -w -s 0x0000 -p {polarity} -d {trigger_bit_delay} -a 0x0001 --start_DAQ_pulse --stop_DAQ_pulse --check_valid_data_start".split())
IPC_queue = multiprocessing.Queue()
process = multiprocessing.Process(target=run_script.main_process, args=(IPC_queue, options, f'main_process_Start_LEDs_Board0_selfTrigger'))
process.start()

IPC_queue.put('memoFC Start Triggerbit QInj BCR')
while not IPC_queue.empty():
    pass
time.sleep(10)
IPC_queue.put('stop DAQ')
IPC_queue.put('memoFC Triggerbit')
while not IPC_queue.empty():
    pass
IPC_queue.put('allow threads to exit')
process.join()

del IPC_queue, process, parser

### Board 1

In [ ]:
for inum in range(475, 490, 1):
    trigger_bit_delay = inum
    print(inum)
    parser = parser_arguments.create_parser()
    (options, args) = parser.parse_args(args=f"-f --useIPC --hostname {hostname} -t 20 -o CanBeRemoved_Board1_LinkCheck_selfTrigger_delay{inum} -v -w -s 0x0004 -p {polarity} -d {int('001011'+format(inum, '010b'), base=2)} -a 0x0022".split())
    IPC_queue = multiprocessing.Queue()
    process = multiprocessing.Process(target=run_script.main_process, args=(IPC_queue, options, f'main_process_Start_LEDs_Board1_LinkCheck_selfTrigger_delay{inum}'))
    process.start()

    IPC_queue.put('memoFC Start Triggerbit QInj BCR')
    while not IPC_queue.empty():
        pass
    time.sleep(10)
    IPC_queue.put('stop DAQ')
    IPC_queue.put('memoFC Triggerbit')
    while not IPC_queue.empty():
        pass
    IPC_queue.put('allow threads to exit')
    process.join()

    del IPC_queue, process, parser

In [ ]:
for inum in range(475, 490, 1):
    trigger_bit_delay = inum
    print(inum)
    parser = parser_arguments.create_parser()
    (options, args) = parser.parse_args(args=f"-f --useIPC --hostname {hostname} -t 20 -o CanBeRemoved_Board1_LinkCheck_selfTrigger_delay{inum} -v -w -s 0x0004 -p {polarity} -d {int('001011'+format(inum, '010b'), base=2)} -a 0x00bb".split())
    IPC_queue = multiprocessing.Queue()
    process = multiprocessing.Process(target=run_script.main_process, args=(IPC_queue, options, f'main_process_Start_LEDs_Board1_LinkCheck_selfTrigger_delay{inum}'))
    process.start()

    IPC_queue.put('memoFC Start Triggerbit QInj BCR')
    while not IPC_queue.empty():
        pass
    time.sleep(10)
    IPC_queue.put('stop DAQ')
    IPC_queue.put('memoFC Triggerbit')
    while not IPC_queue.empty():
        pass
    IPC_queue.put('allow threads to exit')
    process.join()

    del IPC_queue, process, parser

#### test delay from previous cell

In [ ]:
trigger_bit_delay = int('001011'+format(484, '010b'), base=2)
parser = parser_arguments.create_parser()
(options, args) = parser.parse_args(args=f"-f --useIPC --hostname {hostname} -t 30 -o CanBeRemoved_Board1_selfTrigger -v -w -s 0x0004 -p {polarity} -d {trigger_bit_delay} -a 0x0022".split())
IPC_queue = multiprocessing.Queue()
process = multiprocessing.Process(target=run_script.main_process, args=(IPC_queue, options, f'main_process_Start_LEDs_Board2_selfTrigger'))
process.start()

IPC_queue.put('memoFC Start Triggerbit QInj BCR')
while not IPC_queue.empty():
    pass
time.sleep(20)
IPC_queue.put('stop DAQ')
IPC_queue.put('memoFC Triggerbit')
while not IPC_queue.empty():
    pass
IPC_queue.put('allow threads to exit')
process.join()

del IPC_queue, process, parser

### Board 2

In [ ]:
for inum in range(475, 490, 1):
    trigger_bit_delay = inum
    print(inum)
    parser = parser_arguments.create_parser()
    (options, args) = parser.parse_args(args=f"-f --useIPC --hostname {hostname} -t 20 -o CanBeRemoved_Board2_LinkCheck_selfTrigger_delay{inum} -v -w -s 0x0008 -p {polarity} -d {int('010011'+format(inum, '010b'), base=2)} -a 0x0044".split())
    IPC_queue = multiprocessing.Queue()
    process = multiprocessing.Process(target=run_script.main_process, args=(IPC_queue, options, f'main_process_Start_LEDs_Board2_LinkCheck_selfTrigger_delay{inum}'))
    process.start()

    IPC_queue.put('memoFC Start Triggerbit QInj BCR')
    while not IPC_queue.empty():
        pass
    time.sleep(10)
    IPC_queue.put('stop DAQ')
    IPC_queue.put('memoFC Triggerbit')
    while not IPC_queue.empty():
        pass
    IPC_queue.put('allow threads to exit')
    process.join()

    del IPC_queue, process, parser

#### test delay from previous cell

In [ ]:
trigger_bit_delay = int('010011'+format(485, '010b'), base=2)
parser = parser_arguments.create_parser()
(options, args) = parser.parse_args(args=f"-f --useIPC --hostname {hostname} -t 30 -o CanBeRemoved_Board2_selfTrigger -v -w -s 0x0008 -p {polarity} -d {trigger_bit_delay} -a 0x0044".split())
IPC_queue = multiprocessing.Queue()
process = multiprocessing.Process(target=run_script.main_process, args=(IPC_queue, options, f'main_process_Start_LEDs_Board2_selfTrigger'))
process.start()

IPC_queue.put('memoFC Start Triggerbit QInj BCR')
while not IPC_queue.empty():
    pass
time.sleep(20)
IPC_queue.put('stop DAQ')
IPC_queue.put('memoFC Triggerbit')
while not IPC_queue.empty():
    pass
IPC_queue.put('allow threads to exit')
process.join()

del IPC_queue, process, parser

### Board 3

In [ ]:
for inum in range(475, 490, 1):
    trigger_bit_delay = inum
    print(inum)
    parser = parser_arguments.create_parser()
    (options, args) = parser.parse_args(args=f"-f --useIPC --hostname {hostname} -t 20 -o CanBeRemoved_Board3_LinkCheck_selfTrigger_delay{inum} -v -w -s 0x000C -p {polarity} -d {int('100011'+format(inum, '010b'), base=2)} -a 0x0088".split())
    IPC_queue = multiprocessing.Queue()
    process = multiprocessing.Process(target=run_script.main_process, args=(IPC_queue, options, f'main_process_Start_LEDs_Board3_LinkCheck_selfTrigger_delay{inum}'))
    process.start()

    IPC_queue.put('memoFC Start Triggerbit QInj BCR')
    while not IPC_queue.empty():
        pass
    time.sleep(10)
    IPC_queue.put('stop DAQ')
    IPC_queue.put('memoFC Triggerbit')
    while not IPC_queue.empty():
        pass
    IPC_queue.put('allow threads to exit')
    process.join()

    del IPC_queue, process, parser

#### test delay from previous cell

In [ ]:
trigger_bit_delay = int('100011'+format(485, '010b'), base=2)
parser = parser_arguments.create_parser()
(options, args) = parser.parse_args(args=f"-f --useIPC --hostname {hostname} -t 30 -o CanBeRemoved_Board3_selfTrigger -v -w -s 0x0004 -p {polarity} -d {trigger_bit_delay} -a 0x0088".split())
IPC_queue = multiprocessing.Queue()
process = multiprocessing.Process(target=run_script.main_process, args=(IPC_queue, options, f'main_process_Start_LEDs_Board3_selfTrigger'))
process.start()

IPC_queue.put('memoFC Start Triggerbit QInj BCR')
while not IPC_queue.empty():
    pass
time.sleep(20)
IPC_queue.put('stop DAQ')
IPC_queue.put('memoFC Triggerbit')
while not IPC_queue.empty():
    pass
IPC_queue.put('allow threads to exit')
process.join()

del IPC_queue, process, parser

## Multiple Boards

In [ ]:
trigger_bit_delay = int('100011'+format(484, '010b'), base=2)
parser = parser_arguments.create_parser()
(options, args) = parser.parse_args(args=f"-f --useIPC --hostname {hostname} -t 30 -o CanBeRemoved_Board13_selfTrigger -v -w -s 0x000C -p {polarity} -d {trigger_bit_delay} -a 0x00bb --counter_duration 0x0001".split())
IPC_queue = multiprocessing.Queue()
process = multiprocessing.Process(target=run_script.main_process, args=(IPC_queue, options, f'main_process_Start_LEDs_Board0_selfTrigger'))
process.start()

IPC_queue.put('memoFC Start Triggerbit QInj BCR')
while not IPC_queue.empty():
    pass
time.sleep(20)
IPC_queue.put('stop DAQ')
IPC_queue.put('memoFC Triggerbit')
while not IPC_queue.empty():
    pass
IPC_queue.put('allow threads to exit')
process.join()

del IPC_queue, process, parser

In [ ]:
trigger_bit_delay = int('001011'+format(485, '010b'), base=2)
parser = parser_arguments.create_parser()
(options, args) = parser.parse_args(args=f"-f --useIPC --hostname {hostname} -t 30 -o CanBeRemoved_Board13_selfTrigger -v -w -s 0x0004 -p {polarity} -d {trigger_bit_delay} -a 0x00bb --counter_duration 0x0001".split())
IPC_queue = multiprocessing.Queue()
process = multiprocessing.Process(target=run_script.main_process, args=(IPC_queue, options, f'main_process_Start_LEDs_Board0_selfTrigger'))
process.start()

IPC_queue.put('memoFC Start Triggerbit QInj BCR')
while not IPC_queue.empty():
    pass
time.sleep(20)
IPC_queue.put('stop DAQ')
IPC_queue.put('memoFC Triggerbit')
while not IPC_queue.empty():
    pass
IPC_queue.put('allow threads to exit')
process.join()

del IPC_queue, process, parser

In [ ]:
trigger_bit_delay = int('000111'+format(485, '010b'), base=2)
parser = parser_arguments.create_parser()
(options, args) = parser.parse_args(args=f"-f --useIPC --hostname {hostname} -t 30 -o CanBeRemoved_Board0_selfTrigger_QInj_Readout013 -v -w -s 0x0000 -p {polarity} -d {trigger_bit_delay} -a 0x000b".split())
IPC_queue = multiprocessing.Queue()
process = multiprocessing.Process(target=run_script.main_process, args=(IPC_queue, options, f'main_process_Start_LEDs_Board0_selfTrigger'))
process.start()

IPC_queue.put('memoFC Start Triggerbit QInj BCR')
while not IPC_queue.empty():
    pass
time.sleep(20)
IPC_queue.put('stop DAQ')
IPC_queue.put('memoFC Triggerbit')
while not IPC_queue.empty():
    pass
IPC_queue.put('allow threads to exit')
process.join()

del IPC_queue, process, parser

In [ ]:
trigger_bit_delay = int('000111'+format(485, '010b'), base=2)
parser = parser_arguments.create_parser()
(options, args) = parser.parse_args(args=f"-f --useIPC --hostname {hostname} -t 320 -o CanBeRemoved_Board0_selfTrigger_Readout013 -v -w -s 0x0000 -p {polarity} -d {trigger_bit_delay} -a 0x000b --check_valid_data_start --clear_fifo".split())
IPC_queue = multiprocessing.Queue()
process = multiprocessing.Process(target=run_script.main_process, args=(IPC_queue, options, f'main_process_Start_LEDs_Board0_selfTrigger'))
process.start()

IPC_queue.put('memoFC Start Triggerbit BCR')
while not IPC_queue.empty():
    pass
time.sleep(300)
IPC_queue.put('stop DAQ')
IPC_queue.put('memoFC Triggerbit')
while not IPC_queue.empty():
    pass
IPC_queue.put('allow threads to exit')
process.join()

del IPC_queue, process, parser

In [ ]:
trigger_bit_delay = int('100111'+format(485, '010b'), base=2)
parser = parser_arguments.create_parser()
(options, args) = parser.parse_args(args=f"-f --useIPC --hostname {hostname} -t 30 -o CanBeRemoved_Board03_selfTrigger -v -w -s 0x0000 -p {polarity} -d {trigger_bit_delay} -a 0x00bb".split())
IPC_queue = multiprocessing.Queue()
process = multiprocessing.Process(target=run_script.main_process, args=(IPC_queue, options, f'main_process_Start_LEDs_Board0_selfTrigger'))
process.start()

IPC_queue.put('memoFC Start Triggerbit QInj BCR')
while not IPC_queue.empty():
    pass
time.sleep(20)
IPC_queue.put('stop DAQ')
IPC_queue.put('memoFC Triggerbit')
while not IPC_queue.empty():
    pass
IPC_queue.put('allow threads to exit')
process.join()

del IPC_queue, process, parser

# Disconnect I2C Device

In [ ]:
del i2c_conn